In [2]:
import os
import sys; sys.path.append("~/HHbbgg_ETH/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)

Welcome to ROOTaaS 6.05/02


<module 'training_utils' from '/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/Training/python/training_utils.pyc'>

In [3]:
ntuples = '20170526'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
signal = [s for s in files if "NonRes" in s]
diphotonJets = [s for s in files if "DiPhotonJets" in s]
gJets_lowPt = [s for s in files if "GJet_Pt-20to40" in s]
gJets_highPt = [s for s in files if "GJet_Pt-40" in s]

utils.IO.add_signal(ntuples,signal,1)
utils.IO.add_background(ntuples,diphotonJets,-1)
utils.IO.add_background(ntuples,gJets_lowPt,-1)
utils.IO.add_background(ntuples,gJets_highPt,-1)

for i in range(len(utils.IO.backgroundName)):        
    print "using background file n."+str(i)+": "+utils.IO.backgroundName[i]
for i in range(len(utils.IO.signalName)):    
    print "using signal file n."+str(i)+": "+utils.IO.signalName[i]

env: data=/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170526
using background file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170526/output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root
using background file n.1: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170526/output_GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.2: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170526/output_GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using signal file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170526/AllNonResSig_20170526.root


In [4]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,DiJetDiPho_DR'.split(",")
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()'.split(",")
branch_names = [c.strip() for c in branch_names]
#branch_names = (b.replace(" ", "_") for b in branch_names)
#branch_names = list(b.replace("-", "_") for b in branch_names)
print branch_names

import pandas as pd
import root_pandas as rpd

#utils.IO.get_cross_sections('jsons/datasets80X_Moriond.json')
#utils.IO.w_sig.append(utils.IO.get_weight('signal','nrSM',0))
#utils.IO.w_bkg(utils.IO.get_weight('background','DiPhoJets',0))
#utils.IO.w_bkg(utils.IO.get_weight('background','GJets',0))
#utils.IO.w_bkg(utils.IO.get_weight('background','GJets',1))


utils.preprocessing.set_signals_and_backgrounds("bbggSelectionTree",branch_names)
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=utils.preprocessing.set_variables(branch_names)

#relative weighting between components of one class is kept, all classes normalized to the same
#weights_sig=utils.preprocessing.weight_signal_with_resolution(weights_sig,y_sig)
weights_sig=utils.preprocessing.weight_signal_with_resolution(weights_sig,y_sig)
weights_bkg,weights_sig=utils.preprocessing.normalize_process_weights(weights_bkg,y_bkg,weights_sig,y_sig)

X_bkg,y_bkg,weights_bkg = utils.preprocessing.randomize(X_bkg,y_bkg,weights_bkg)
X_sig,y_sig,weights_sig = utils.preprocessing.randomize(X_sig,y_sig,weights_sig)

print X_bkg.shape
print y_bkg.shape
#bbggTrees have by default signal and CR events, let's be sure that we clean it
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=utils.preprocessing.clean_signal_events(X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig)
print X_bkg.shape
print y_bkg.shape

y_total_train = utils.preprocessing.get_total_training_sample(y_sig,y_bkg).ravel()
X_total_train = utils.preprocessing.get_total_training_sample(X_sig,X_bkg)

y_total_test = utils.preprocessing.get_total_test_sample(y_sig,y_bkg).ravel()
X_total_test = utils.preprocessing.get_total_test_sample(X_sig,X_bkg)

w_total_train = utils.preprocessing.get_total_training_sample(weights_sig,weights_bkg).ravel()
w_total_test = utils.preprocessing.get_total_test_sample(weights_sig,weights_bkg).ravel()

['leadingJet_bDis', 'subleadingJet_bDis', 'noexpand:fabs(CosThetaStar_CS)', 'noexpand:fabs(CosTheta_bb)', 'noexpand:fabs(CosTheta_gg)', 'noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M()', 'noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()', 'customLeadingPhotonIDMVA', 'customSubLeadingPhotonIDMVA', 'leadingPhotonSigOverE', 'subleadingPhotonSigOverE', 'sigmaMOverMDecorr', 'DiJetDiPho_DR']
in shuffle
(2254174, 13)
(2254174, 1)
(1922962, 13)
(1922962, 1)


In [5]:
import xgboost as xgb
clf = xgb.XGBClassifier(max_depth=5,learning_rate=0.1,n_estimators=50, min_child_weight=1e-5, nthread= 12)

In [6]:
reload(utils)
param_grid = {"n_estimators": [1000,1500],
              "max_depth": [3,4,5],                                                                                                                                                                                                
              'reg_lambda':[0.1, 1]
              }
#utils.optimization.optimize_parameters_randomizedCV(clf,X_total_train,y_total_train,param_grid,cvOpt=5,nIter=500,weights=w_total_train)
utils.optimization.optimize_parameters_randomizedCV(clf,X_total_train,y_total_train,param_grid,cvOpt=5,weights=w_total_train)

=====Optimization with randomized search cv=====
-Initial Accuracy-
Accuracy: 0.95357 (+/- 0.00067)
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/Training/python/training_utils.py:642: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if weights == None:
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed: 24.1min
[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed: 34.5min finished


Best parameter set found on development set:

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1e-05, missing=None, n_estimators=1500, nthread=12,
       objective='binary:logistic', reg_alpha=0, reg_lambda=0.1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

Grid scores on a subset of the development set:

0.9532 (+/-0.0007) for {'n_estimators': 1000, 'reg_lambda': 0.1, 'max_depth': 3}
0.9463 (+/-0.0006) for {'n_estimators': 1000, 'reg_lambda': 1, 'max_depth': 3}
0.9536 (+/-0.0007) for {'n_estimators': 1500, 'reg_lambda': 0.1, 'max_depth': 3}
0.9543 (+/-0.0007) for {'n_estimators': 1500, 'reg_lambda': 0.1, 'max_depth': 5}
0.9535 (+/-0.0008) for {'n_estimators': 1000, 'reg_lambda': 0.1, 'max_depth': 4}
0.9490 (+/-0.0008) for {'n_estimators': 1500, 'reg_lambda': 1, 'max_depth': 5}
0.9541 (+/-0.0007) for {'n_estimators': 1500, 'reg_lambda': 0.1, 'max_depth': 4}
0.

/swshare/anaconda/lib/python2.7/site-packages/sklearn/model_selection/_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/swshare/anaconda/lib/python2.7/site-packages/sklearn/model_selection/_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.95318, std: 0.00071, params: {'n_estimators': 1000, 'reg_lambda': 0.1, 'max_depth': 3},
 mean: 0.94635, std: 0.00065, params: {'n_estimators': 1000, 'reg_lambda': 1, 'max_depth': 3},
 mean: 0.95360, std: 0.00067, params: {'n_estimators': 1500, 'reg_lambda': 0.1, 'max_depth': 3},
 mean: 0.95428, std: 0.00069, params: {'n_estimators': 1500, 'reg_lambda': 0.1, 'max_depth': 5},
 mean: 0.95352, std: 0.00076, params: {'n_estimators': 1000, 'reg_lambda': 0.1, 'max_depth': 4},
 mean: 0.94903, std: 0.00076, params: {'n_estimators': 1500, 'reg_lambda': 1, 'max_depth': 5},
 mean: 0.95405, std: 0.00070, params: {'n_estimators': 1500, 'reg_lambda': 0.1, 'max_depth': 4},
 mean: 0.94902, std: 0.00075, params: {'n_estimators': 1500, 'reg_lambda': 1, 'max_depth': 4},
 mean: 0.95373, std: 0.00073, params: {'n_estimators': 1000, 'reg_lambda': 0.1, 'max_depth': 5},
 mean: 0.94647, std: 0.00065, params: {'n_estimators': 1000, 'reg_lambda': 1, 'max_depth': 5}]